In [142]:
import os
import pandas as pd
import ast

rootdir = "C:/Users/hp/Desktop/Sproj/Active"  ##Set Root folder directory.

In [143]:

all_dicts = []
all_dicts_normal=[]
for root,dirs,files in os.walk(rootdir):
    new_dict = {}
    dictionary_made = False
    for name in files:
        new_path = os.path.join(root,'stream_details.txt')
        if not os.path.exists(new_path):
            continue
            
        if not dictionary_made:
            with open(new_path) as data:
                temp_dict = ast.literal_eval(data.read())
                for key,values in temp_dict.items():
                    new_key = key.strip()
                    new_dict[new_key]=values
            dictionary_made = True
            
        if name.split('.')[1] == 'txt' or name.split('.')[1] == 'py' or name.split('.')[1]=='exe' or name.split('.')[1]=='ipynb':
            continue
        else:
            file_path = os.path.join(root,name)
            name=name.replace(".mp4",'')
            ad_id = name.split('_')
            if(ad_id[0]=="MainVideo"):
                new_dict['Main_Video']["Size"]=os.path.getsize(file_path)
            else:
                if(ad_id[1]==''):
                    new_id = "_"+ad_id[2]
                    stripped_key = new_id.strip()
                    new_dict[stripped_key]['Size']=os.path.getsize(file_path)
                else:
                    stripped_key = ad_id[1].strip()
                    new_dict[stripped_key]['Size']=os.path.getsize(file_path)
    
    temp_dict={}
    temp_dict[root[-1]]=new_dict
    all_dicts_normal.append(new_dict)
    all_dicts.append(temp_dict)

In [144]:
#Making Rows for DataFrame

all_dicts=all_dicts[1:] #Removing Empty dict at index 0 since it is of the root folder.
all_dicts_normal=all_dicts_normal[1:]
max_number_of_ads = -1

for i in range(len(all_dicts)):
    for key,value in all_dicts[i].items():
        if value["Main_Video"]['UniqueAds']>max_number_of_ads:
            max_number_of_ads=value['Main_Video']['UniqueAds']

column_names = ["Main_Video_Url","Unique_Ads","Duration(s)","Size(Bytes)","Avg_Buffer","Resolution"]
add_columns=[]

for i in range(max_number_of_ads):
    ad_id = f"Advertisement {i+1} ID"
    add_columns.append(ad_id)
    ad_count=f"Advertisement {i+1} Count"
    add_columns.append(ad_count)
    ad_skippable = f"Advertisement {i+1} Skippable"
    add_columns.append(ad_skippable)
    skip_duration = f"Advertisement {i+1} Skip Duration"
    add_columns.append(skip_duration)
    resolution = f"Advertisement {i+1} Resolution"
    add_columns.append(resolution)
    size = f"Advertisement {i+1} Size(Bytes)"
    add_columns.append(size)

new_columns = column_names+add_columns

In [145]:
#Generate the rows
all_data_rows=[]
for i in range(len(all_dicts_normal)):
    row=[]
    main_video_url = all_dicts_normal[i]['Main_Video']['Url']
    row.append(main_video_url)
    unique_ads = all_dicts_normal[i]['Main_Video']['UniqueAds']
    row.append(unique_ads)
    duration= all_dicts_normal[i]['Main_Video']['Total Duration']
    row.append(duration)
    size=all_dicts_normal[i]["Main_Video"]["Size"]
    row.append(size)
    avg= all_dicts_normal[i]['Main_Video']['AvgBuffer']
    row.append(avg)
    res= all_dicts_normal[i]['Main_Video']['Resolution']
    row.append(res)
    for key,value in all_dicts_normal[i].items():
        if key == "Main_Video":
            continue
        else:
            row.append(key)
            row.append(value['Count'])
            row.append(value['Skippable'])
            row.append(value['SkipDuration'])
            row.append(value['Resolution'])
            row.append(value['Size'])
    all_data_rows.append(row)

In [146]:
df = pd.DataFrame(all_data_rows,columns=new_columns)

In [147]:
df

,Main_Video_Url,Unique_Ads,Duration(s),Size(Bytes),Avg_Buffer,Resolution,Advertisement 1 ID,Advertisement 1 Count,Advertisement 1 Skippable,Advertisement 1 Skip Duration,...,Advertisement 3 Skippable,Advertisement 3 Skip Duration,Advertisement 3 Resolution,Advertisement 3 Size(Bytes),Advertisement 4 ID,Advertisement 4 Count,Advertisement 4 Skippable,Advertisement 4 Skip Duration,Advertisement 4 Resolution,Advertisement 4 Size(Bytes)
0,https://www.youtube.com/watch?v=kXpOEzNZ8hQ,4,281.000,6991173,0.722134,854x480@24,EDuJoFs6VOs,1.0,1.0,1.0,...,1.0,3.0,854x480@25,2207556.0,OJZWVifQLPw,1.0,1.0,3.0,854x480@30,17892250.0
1,https://www.youtube.com/watch?v=UdiaQcAjzT0,0,164.881,12120194,0.830543,854x480@30,None,NaN,NaN,NaN,...,NaN,NaN,None,NaN,None,NaN,NaN,NaN,None,NaN
2,https://www.youtube.com/watch?v=a4YwJCZRh5M,4,205.000,5692729,0.795211,360x360@25,A1xfsJkv6gk,1.0,1.0,1.0,...,1.0,4.0,854x480@25,13846335.0,FgmwOARuEGE,1.0,1.0,3.0,854x480@24,23366427.0
3,https://www.youtube.com/watch?v=PCupTKYKpmE,2,264.000,3686469,0.748192,360x360@25,_FCwqB7wJBA,1.0,1.0,1.0,...,NaN,NaN,None,NaN,None,NaN,NaN,NaN,None,NaN
4,https://www.youtube.com/watch?v=jW-bkbSWb4A,0,199.301,18510668,0.844211,854x480@24,None,NaN,NaN,NaN,...,NaN,NaN,None,NaN,None,NaN,NaN,NaN,None,NaN
5,https://www.youtube.com/watch?v=QhRM-X9kqFs,4,790.000,47800968,0.403981,854x480@30,SUp1FfRTQDM,1.0,1.0,1.0,...,1.0,3.0,854x480@24,5647509.0,xIyz9NV-4ww,1.0,1.0,5.0,854x480@25,2207556.0
